In [1]:
import pysam

from pysam import VariantFile

import pandas as pd

from pyliftover import LiftOver

## Variant calling from bcf:

In [11]:
sample = "I13518"
#bcf_route = "./arslantepe/" + sample + "/" + sample + "_Y"
bcf_route = "/home/javi/Documents/genomics/samples/southern_arc/" + sample + "/" + sample + "_Y"

bcf_in = VariantFile(bcf_route + ".bcf")
#bcf_in = VariantFile(bcf_route + ".bcf");  # auto-detect input format
#bcf_out = VariantFile('-', 'w', header=bcf_in.header)

#for rec in bcf_in.fetch():
#    bcf_out.write(rec)

[E::idx_find_and_load] Could not retrieve index file for '/home/javi/Documents/genomics/samples/southern_arc/I13518/I13518_Y.bcf'


In [ ]:
df_0 = pd.read_csv("./genoma_prueba_all_calls.vcf", sep='\t', comment='#');
df_0.head()

In [3]:
# I12162: J-Z2177(xPH185, FGC33287, FT50020, FT51478)
# I3983: J-Z1847(xZ500, Z502, Z508, CTS3261, SK1336, Z462, Z7664, Z7658, FT8384, Z6271, Z28583, Y18824, Z488, PH4809, A25797, Z40570, Z40562, Z40560, PF7402, Z517)

In [3]:
# Regiones a excluir:
par1 = [1,2781479];
cen = [10072350,11686750];
dyz19 = [20054914,20351054];
ppr = [26637971,26673210];
par2 = [56887903,57217415];
mutations_to_exclude = ["C to T", "A to T", "G to C", "A to G"];

In [ ]:
print(bcf_in.header)

In [4]:
def check_region(position):
    if (par1[0] <= position <= par1[1]):
        i_reg = 'par1';
    elif (cen[0] <= position <= cen[1]):
        i_reg = 'cen';
    elif (dyz19[0] <= position <= dyz19[1]):
        i_reg = 'dyz19';
    elif (ppr[0] <= position <= ppr[1]):
        i_reg = 'ppr';
    elif (par2[0] <= position <= par2[1]):
        i_reg = 'par2';
    else:
        i_reg = '';
    return i_reg
def check_mutation(reference, genotype):
    i_mut = '';
    if (reference == 'C') & (genotype == 'T'):
        i_mut = 'C to T deamination';
    elif (reference == 'A') & (genotype == 'T'):
        i_mut = 'A to T risk of strand misidentification';
    elif (reference == 'G') & (genotype == 'C'):
        i_mut = 'G to C risk of strand misidentification';
    elif (reference == 'G') & (genotype == 'A'):
        i_mut = 'G to A deamination';
    return i_mut

### Cambiando a hg38 y anotando regiones:

In [5]:
# Cambiando a hg38 (sin excluir y anotando):
vc_list = [];
lo = LiftOver('hg19', 'hg38');
for rec in bcf_in.fetch():
    i_pos_0 = lo.convert_coordinate('chrY', int(rec.pos));
    if i_pos_0 != []:
        # Anotamos las posiciones no adecuadas para filogenia:
        i_pos = int(i_pos_0[0][1]);            
        i_reg = check_region(i_pos);
        # Extraemos el genotipo:
        i_gen_0 = rec.alts;
        i_ref = rec.ref;
        i_mut = '';
        if i_gen_0 == None:
            i_gen = i_ref;
        else:
            i_gen = i_gen_0[0];
            i_mut = check_mutation(i_ref, i_gen);
        # Creamos la lista:
        vc_list.append([i_pos, rec.ref, rec.alts, i_gen, i_mut, i_reg, rec.qual]);
    else:
        vc_list.append([int(0), rec.ref, rec.alts, rec.alts, '', '', rec.qual]);

In [7]:
len(vc_list)

744699

In [5]:
chromosomas_contenidos = [];
for rec in bcf_in.fetch():
        chromosomas_contenidos.append(rec.chrom);
set(chromosomas_contenidos)

{'Y'}

### Cambiando a hg38 excluyendo regiones:

In [ ]:
# Cambiando a hg38:
vc_list = [];
lo = LiftOver('hg19', 'hg38');
for rec in bcf_in.fetch():
    i_pos_0 = lo.convert_coordinate('chrY', int(rec.pos));
    if i_pos_0 != []:    
        i_pos = int(i_pos_0[0][1]);
        if ~((par1[0] <= i_pos <= par1[1]) | (cen[0] <= i_pos <= cen[1]) | (dyz19[0] <= i_pos <= dyz19[1]) |
           (ppr[0] <= i_pos <= ppr[1]) | (par2[0] <= i_pos <= par2[1])):
            vc_list.append([i_pos, rec.ref, rec.alts, rec.qual]);
    else:
        vc_list.append([int(0), rec.ref, rec.alts, rec.qual]);

### Sin convertir excluyendo regiones:

In [13]:
# Sin convertir:
vc_list = [];
for rec in bcf_in.fetch():
    if rec.chrom:
        i_pos_0 = int(rec.pos);
        if ~((par1[0] <= i_pos <= par1[1]) | (cen[0] <= i_pos <= cen[1]) | (dyz19[0] <= i_pos <= dyz19[1]) |
        (ppr[0] <= i_pos <= ppr[1]) | (par2[0] <= i_pos <= par2[1])):
            vc_list.append([i_pos, rec.ref, rec.alts, rec.qual]);

SyntaxError: invalid syntax (1364058564.py, line 4)

### Sin convertir sin excluir regiones:

In [8]:
# Sin convertir:
vc_list = [];
for rec in bcf_in.fetch():
    if rec.chrom:
        i_pos = int(rec.pos);
        vc_list.append([i_pos, rec.ref, rec.alts, rec.qual]);

In [21]:
len(vc_list)

154

### Generamos un .csv anotado con las variantes:

In [6]:
# Generamos un dataframe auxiliar a partir del bcf:
df = pd.DataFrame(vc_list, columns =['Position', 'Ref', 'Alt', 'Genotype', 'problematic_mutation', 'problematic_region', 'Qual'], dtype = str);
df = df.rename(columns={'Position': 'start'});
df = df.astype({"start": int});

In [7]:
# Yleaf:
anotacion = "yleaf"
df_yleaf = pd.read_csv('SNP_annotations_hg38/yleaf_WGS_hg38.txt', sep="\t", header=None)
df_yleaf.columns = ['chr', 'Name', 'ISOGG_haplogroup', 'start', 'mutation', 'allele_anc', 'allele_der'];

#Ybrowse:
df_ybrowse = pd.read_csv('SNP_annotations_hg38/snps_hg38.csv')[(['seqid', 'Name', 'ISOGG_haplogroup', 'start', 'mutation', 'allele_anc', 'allele_der'])];
df_ybrowse = df_ybrowse.rename(columns={'seqid': 'chr'});

# Seleccionamos todos los de Yleaf en el sample más los de Ybrowse que están en el sample pero no en Yleaf:
df_int_yleaf = df_yleaf[df_yleaf["start"].isin(df["start"].tolist())]
df_int_ybrowse_no_yleaf = df_ybrowse[(df_ybrowse["start"].isin(df["start"].tolist())) & ~(df_ybrowse["start"].isin(df_int_yleaf["start"].tolist()))]
df_anotation = pd.concat([df_int_yleaf, df_int_ybrowse_no_yleaf])
df_anotation.head()

,chr,Name,ISOGG_haplogroup,start,mutation,allele_anc,allele_der
0,chry,L984,A0,7193464,G->T,G,T
1,chry,V225,A0,7728274,C->T,C,T
2,chry,V165,A0,14164925,G->A,G,A
3,chry,L1012,A0,19319543,C->T,C,T
4,chry,L1001,A0,14280720,G->T,G,T


In [8]:
# Mezclamos:
df_final = pd.merge(df, df_anotation, on='start')
# En el archivo de ybrowse hay algunas posiciones con distinta referencia (quizá por backmutación)
# nos quedamos solo con las que coinciden con nuestra referencia:
df_final = df_final[df_final['Ref'] == df_final['allele_anc']]
#mutations_to_exclude = ["C to T", "A to T", "G to C", "A to G"];
#df_final = df_final[~df_final["mutation"].isin(mutations_to_exclude)];
df_final = df_final.sort_values(by=['Qual'], ascending=False)
# Guardamos:
df_final.to_csv(bcf_route + "_annotated_" + anotacion + ".csv", encoding='utf-8', index=False)
df_final.head()

,start,Ref,Alt,Genotype,problematic_mutation,problematic_region,Qual,chr,Name,ISOGG_haplogroup,mutation,allele_anc,allele_der
271507,17449023,A,None,A,,,99.0,chrY,Y191431,R-U106,A to C,A,C
260057,17014314,G,None,G,,,99.0,chrY,FGC49870,unknown,G to T,G,T
153537,12282345,A,None,A,,,99.0,chrY,Y49486,J2,A to G,A,G
240740,16018130,T,None,T,,,99.0,chrY,JFS0371,R1b,T to A,T,A
240741,16018130,T,None,T,,,99.0,chrY,BY50396,unknown,T to A,T,A


In [66]:
df_final[df_final["Name"] == "Z6065"]

,start,Ref,Alt,Genotype,problematic_mutation,problematic_region,Qual,chr,Name,ISOGG_haplogroup,mutation,allele_anc,allele_der
41203,16070426,G,None,G,,,148.0,chry,Z6065,J2a1a1a2a,G->T,G,T


In [15]:
set(df_final["Alt"].tolist())

{('A',), ('C',), ('G',), ('T',), ('TTC',), None}

In [28]:
df_final_variants = df_final[~(df_final["Alt"].isnull())]

In [29]:
len(df_final_variants)

44

In [30]:
df_final_variants.to_csv(bcf_route + "_annotated_variants_" + anotacion + ".csv", encoding='utf-8');

In [31]:
df_final = df_final_variants

### Preparamos un .txt con las posiciones observadas para meter en CladeFinder:

In [9]:
basic_haplogroup_prediction = "R"
posiciones_problematicas = ['par1', 'cen', 'dyz19', 'ppr', 'par2'];
#mutaciones_problematicas = ['C to T deamination', 'A to T risk of strand misidentification', 'G to C risk of strand misidentification', 'G to A deamination'];
mutaciones_problematicas = ['C to T deamination', 'G to A deamination'];
# Filtramos las posiciones ancestrales para que correspondan a la predicción basica:
cond_anc_in_pred = (df_final["ISOGG_haplogroup"].str.contains(basic_haplogroup_prediction)) & (df_final["Alt"].isnull())
cond_der_all = ~(df_final["Alt"].isnull()) & ~(df_final["problematic_mutation"].isin(mutaciones_problematicas))
df_clade_finder = df_final[cond_anc_in_pred | cond_der_all]
# Quitamos las mutaciones problemáticas:
#df_clade_finder = df_clade_finder[~df_clade_finder["problematic_mutation"].isin(mutaciones_problematicas)];
# Quitamos las posiciones problemáticas:
#df_clade_finder = df_clade_finder[~df_clade_finder["problematic_region"].isin(posiciones_problematicas)];
#df_clade_finder = df_clade_finder[df_clade_finder["ISOGG_haplogroup"] != 'unknown'];
#df_clade_finder = df_clade_finder[~df_clade_finder["ISOGG_haplogroup"].str.contains('not listed')];
ref_alleles = df_clade_finder['Ref'].tolist();
genotypes = df_clade_finder['Genotype'].tolist();
snp_names = df_clade_finder['Name'].tolist();
annotated_mutation = df_clade_finder['allele_der'].tolist();
for i in range(0,len(ref_alleles)):
    # Cuando el genotipo es ancestral:
    if ref_alleles[i] == genotypes[i]:
        i_value = snp_names[i] + "-";
        
        with open(bcf_route + "_" + anotacion + '_for_clade_finder.txt', 'a') as f:
            f.write(i_value + ', ');
    # Cuando el genotipo es derivado:
    else:
        if genotypes[i] == annotated_mutation[i]:
            i_value = snp_names[i] + "+";
            with open(bcf_route + "_" + anotacion + '_for_clade_finder.txt', 'a') as f:
                f.write(i_value + ', ');
df_clade_finder.head()

,start,Ref,Alt,Genotype,problematic_mutation,problematic_region,Qual,chr,Name,ISOGG_haplogroup,mutation,allele_anc,allele_der
271507,17449023,A,None,A,,,99.0,chrY,Y191431,R-U106,A to C,A,C
240740,16018130,T,None,T,,,99.0,chrY,JFS0371,R1b,T to A,T,A
325710,20278701,T,None,T,,dyz19,99.0,chrY,BY20296,R,T to C,T,C
325707,20278697,G,None,G,,dyz19,99.0,chrY,BY27404,R,G to C,G,C
325704,20278689,G,None,G,,dyz19,99.0,chrY,BY218856,R,G to T,G,T


In [ ]:
# Solo los positivos

In [10]:
basic_haplogroup_prediction = ""
posiciones_problematicas = ['par1', 'cen', 'dyz19', 'ppr', 'par2'];
#mutaciones_problematicas = ['C to T deamination', 'A to T risk of strand misidentification', 'G to C risk of strand misidentification', 'G to A deamination'];
mutaciones_problematicas = ['C to T deamination', 'G to A deamination'];
# Filtramos las posiciones ancestrales para que correspondan a la predicción basica:
cond_anc_in_pred = (df_final["ISOGG_haplogroup"].str.contains(basic_haplogroup_prediction)) & (df_final["Alt"].isnull())
cond_der_all = ~(df_final["Alt"].isnull()) & ~(df_final["problematic_mutation"].isin(mutaciones_problematicas))
df_clade_finder = df_final[cond_anc_in_pred | cond_der_all]
# Quitamos las mutaciones problemáticas:
#df_clade_finder = df_clade_finder[~df_clade_finder["problematic_mutation"].isin(mutaciones_problematicas)];
# Quitamos las posiciones problemáticas:
#df_clade_finder = df_clade_finder[~df_clade_finder["problematic_region"].isin(posiciones_problematicas)];
#df_clade_finder = df_clade_finder[df_clade_finder["ISOGG_haplogroup"] != 'unknown'];
#df_clade_finder = df_clade_finder[~df_clade_finder["ISOGG_haplogroup"].str.contains('not listed')];
ref_alleles = df_clade_finder['Ref'].tolist();
genotypes = df_clade_finder['Genotype'].tolist();
snp_names = df_clade_finder['Name'].tolist();
annotated_mutation = df_clade_finder['allele_der'].tolist();
for i in range(0,len(ref_alleles)):
    # Cuando el genotipo no es ancestral:
    if not ref_alleles[i] == genotypes[i]:
        if genotypes[i] == annotated_mutation[i]:
            i_value = snp_names[i] + "+";
            with open(bcf_route + "_" + anotacion + '_for_clade_finder_positives.txt', 'a') as f:
                f.write(i_value + ', ');
df_clade_finder.head()

,start,Ref,Alt,Genotype,problematic_mutation,problematic_region,Qual,chr,Name,ISOGG_haplogroup,mutation,allele_anc,allele_der
271507,17449023,A,None,A,,,99.0,chrY,Y191431,R-U106,A to C,A,C
260057,17014314,G,None,G,,,99.0,chrY,FGC49870,unknown,G to T,G,T
153537,12282345,A,None,A,,,99.0,chrY,Y49486,J2,A to G,A,G
240740,16018130,T,None,T,,,99.0,chrY,JFS0371,R1b,T to A,T,A
240741,16018130,T,None,T,,,99.0,chrY,BY50396,unknown,T to A,T,A


In [49]:
df_clade_finder[df_clade_finder["Name"] == "Z19867"]

,start,Ref,Alt,Genotype,problematic_mutation,problematic_region,Qual,chr,Name,ISOGG_haplogroup,mutation,allele_anc,allele_der
6428,10200631,C,None,C,,cen,55.0,chrY,Z19867,T (not listed),C to T,C,T


In [80]:
df_clade_finder.to_csv(bcf_route + "_clade_finder_annotated.csv", encoding='utf-8');

In [79]:
len(df_clade_finder)

63072

In [ ]:
#Para copiar al portapapeles desde línea de comandos:
! xclip -sel c < ./alalakh/ALA131b/ALA131b_Y_yleaf_for_clade_finder.txt

In [ ]:
import io
import os
import pandas as pd


def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

## Generating the bcf with samtools:

In [ ]:
samtools index ART022.1240K.PE.bam

# Antes que indexar, hay que ordenar el .bam

samtools sort ART022.1240K.PE.bam -o ART022.1240K.PE.sort.bam

samtools index ART022.1240K.PE.sort.bam

samtools view -b ART022.1240K.PE.sort.bam Y > chromosome_Y_ART022.extraction.bam

# Esto no es siempre necesario:
# Para indexar la referencia:
samtools faidx reference.fa

samtools view -H chromosome_Y_ART022.extraction.bam | sed -e 's/SN:1/SN:chr1/' | sed -e 's/SN:2/SN:chr2/' | sed -e 's/SN:3/SN:chr3/' | sed -e 's/SN:4/SN:chr4/' | sed -e 's/SN:5/SN:chr5/' | sed -e 's/SN:6/SN:chr6/' | sed -e 's/SN:7/SN:chr7/' | sed -e 's/SN:8/SN:chr8/' | sed -e 's/SN:9/SN:chr9/' | sed -e 's/SN:10/SN:chr10/' | sed -e 's/SN:11/SN:chr11/' | sed -e 's/SN:12/SN:chr12/' | sed -e 's/SN:13/SN:chr13/' | sed -e 's/SN:14/SN:chr14/' | sed -e 's/SN:15/SN:chr15/' | sed -e 's/SN:16/SN:chr16/' | sed -e 's/SN:17/SN:chr17/' | sed -e 's/SN:18/SN:chr18/' | sed -e 's/SN:19/SN:chr19/' | sed -e 's/SN:20/SN:chr20/' | sed -e 's/SN:21/SN:chr21/' | sed -e 's/SN:22/SN:chr22/' | sed -e 's/SN:X/SN:chrX/' | sed -e 's/SN:Y/SN:chrY/' | sed -e 's/SN:MT/SN:chrM/' | samtools reheader - chromosome_Y_ART022.extraction.bam > test_chrY.bam

# Solo variantes:
bcftools mpileup -Ou -f reference.fa chromosome_Y_ART022.extraction.bam | bcftools call -mv -Ob -o calls.bcf

# Para retener ancestrales:
bcftools mpileup -Ou -f reference.fa chromosome_Y_ART022.extraction.bam | bcftools call -mv -Ob -o calls.bcf

# Para pasar a vcf:

bcftools view Variant_calling_test/ERR4059728_VK346/VK346_calls.bcf > Variant_calling_test/ERR4059728_VK346/VK346_calls.vcf 

In [3]:
pysam.sort("-o", sample_name + "_sort.bam", sample_name + ".bam")

''

In [4]:
pysam.index(sample_name + "_sort.bam")

''

In [39]:
#pysam.view("-b", sample_name + "chromosome_Y_extraction.bam", "Y", sample_name + "_sort.bam")

# terminal_command = "samtools view -b" + sample_name + "_sort.bam" + "Y >" + sample_name + "chromosome_Y.extraction.bam"
#! printf terminal_command

# %%bash
# printf terminal_command

In [6]:
! samtools view -b ART022.1240K.PE_sort.bam Y > ART022.1240K.PE_chromosome_Y_.extraction.bam

In [ ]:
#samfile = pysam.AlignmentFile("ART022.1240K.PE.bam", "rb")
samfile = pysam.AlignmentFile("chromosome_Y_ART022.extraction.bam", "rb")

sam_lista = list(samfile);

sam_lista[:10]

for x in sam_lista[:10]:
    print(x)
    
    # Para que funcione samfile.fetch() hay que ejecutar en terminal:
# samtools index SAMPLE.bam
#for read in samfile.fetch('chr1', 100, 120):
for read in samfile.fetch(100, 120):
     print(read)

In [ ]:
with open('hg19_reference.fa') as topo_file:
    for line in topo_file:
        if "chr" in line:
            line = line.replace("chr", "");
        with open('hg19_modified_reference.fa', 'a') as f:
            f.write(line);